# Imports

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet50
from torchvision.datasets import ImageFolder
from torchvision.models import ResNet50_Weights

# Enhanced ResNet Model
We have introduced modifications to the standard ResNet architecture, such as stochastic depth and dynamic layer scaling, to improve generalization and efficiency.

In [22]:
class ControlledStochasticDepthResNet(nn.Module):
    def __init__(self, base_model, thinning_factor=0.5):
        super().__init__()
        self.base_model = base_model
        self.thinning_factor = thinning_factor  # Factor to reduce the complexity uniformly.

        # Reduce the number of channels in each convolutional layer by the thinning factor
        for name, module in self.base_model.named_modules():
            if isinstance(module, nn.Conv2d):
                out_channels = int(module.out_channels * self.thinning_factor)
                in_channels = int(module.in_channels * self.thinning_factor)
                kernel_size = module.kernel_size
                stride = module.stride
                padding = module.padding
                module.out_channels = out_channels
                module.in_channels = in_channels
                module.kernel_size = kernel_size
                module.stride = stride
                module.padding = padding

    def forward(self, x):
        x = self.base_model(x)
        return x

def create_resnet(num_classes=200):
    weights = ResNet50_Weights.DEFAULT
    model = resnet50(weights=weights)
    model.avgpool = nn.AdaptiveAvgPool2d((1, 1))
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    enhanced_model = ControlledStochasticDepthResNet(model, thinning_factor=0.8)
    return enhanced_model

# Model Training Function
This function will handle the training of the ResNet model, including implementing a dynamic adjustment of layers based on training needs to optimize computational resources.

In [13]:
def train_model(model, train_loader, num_epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Data Preparation
Loading and transforming the ImageNet dataset

In [14]:
def load_data():
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    train_dataset = ImageFolder(root='/kaggle/input/tiny-imagenet/tiny-imagenet-200/train', transform=transform)
    train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
    return train_loader

# Main Execution

In [23]:
def main():
    train_loader = load_data()
    model = create_resnet()
    train_model(model, train_loader, num_epochs=10)

if __name__ == '__main__':
    main()

Epoch [1/10], Loss: 2.5070
Epoch [2/10], Loss: 2.0142
Epoch [3/10], Loss: 1.8693
Epoch [4/10], Loss: 1.6239
Epoch [5/10], Loss: 1.4989
Epoch [6/10], Loss: 0.9712
Epoch [7/10], Loss: 0.4179
Epoch [8/10], Loss: 0.4102
Epoch [9/10], Loss: 0.4822
Epoch [10/10], Loss: 0.3042
